In [4]:
import numpy as np

# Beispielwerte für die Winkel (in Grad) und Beschleunigungen (in g)
pitch_deg = 30   # Neigung um die X-Achse
roll_deg = 20    # Neigung um die Y-Achse
# Angenommene Beschleunigung, wenn der Sensor perfekt aufrecht steht
# Diese Werte sollten die Gravitationskraft reflektieren
acc_x_g = 0
acc_y_g = 0
acc_z_g = 1

# Konvertiere Winkel von Grad in Bogenmaß
pitch = np.radians(pitch_deg)
roll = np.radians(roll_deg)

# Konvertiere Beschleunigungen von g in m/s²
acc_x = acc_x_g * 9.81
acc_y = acc_y_g * 9.81
acc_z = acc_z_g * 9.81

# Berechne die Richtung des Beschleunigungsvektors
# Korrigiere die Beschleunigungswerte für die Neigung des Sensors
acc_x_corrected = acc_x * np.cos(pitch) - acc_z * np.sin(pitch)
acc_y_corrected = acc_x * np.sin(roll) * np.sin(pitch) + acc_y * np.cos(roll) - acc_z * np.sin(roll) * np.cos(pitch)
acc_z_corrected = acc_x * np.cos(roll) * np.sin(pitch) + acc_y * np.sin(roll) + acc_z * np.cos(pitch) * np.cos(roll)

# Berechne die Summe der Gravitationskomponenten
gravity_magnitude = np.sqrt(acc_x_corrected**2 + acc_y_corrected**2 + acc_z_corrected**2)

# Ausgabe der korrigierten Beschleunigungswerte und der Summe der Gravitationskomponenten
print(f"Beschleunigung nach Korrektur (m/s²):")
print(f"X: {acc_x_corrected:.2f}")
print(f"Y: {acc_y_corrected:.2f}")
print(f"Z: {acc_z_corrected:.2f}")

print(f"\nGesamte Gravitationskomponente (m/s²): {gravity_magnitude:.2f}")


Beschleunigung nach Korrektur (m/s²):
X: -4.90
Y: -2.91
Z: 7.98

Gesamte Gravitationskomponente (m/s²): 9.81
